In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Bidirectional, TimeDistributed,Conv1D, MaxPooling1D, Input, concatenate
from keras.layers.recurrent import SimpleRNN
from keras.layers import GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.callbacks import EarlyStopping
import keras
import os
import tarfile
import re
import pandas as pd
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import KFold


Using TensorFlow backend.
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [89]:
def RNN(maxlen = 50, max_features = 4590, embed_size =32):
    model = Sequential()
    model.add(Embedding(max_features, embed_size, input_length=maxlen))
#     model.add(Dropout(0.5))
    model.add(GRU(16,dropout=0.2))
#     model.add(Dense(8, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())

    return model

def CNN(maxlen=50, max_features=4590, embed_size=32):
    # Inputs
    comment_seq = Input(shape=[maxlen], name='x_seq')

    # Embeddings layers
    emb_comment = Embedding(max_features, embed_size)(comment_seq)

    # conv layers
    convs = []
#     filter_sizes = [2, 3, 4, 5]
    filter_sizes = [2, 3]

    for fsz in filter_sizes:
        l_conv = Conv1D(filters=10, kernel_size=fsz, activation='relu')(emb_comment)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    merge = concatenate(convs, axis=1)

    out = Dropout(0.2)(merge)
    output = Dense(16, activation='relu')(out)

    output = Dense(units=1, activation='sigmoid')(output)

    model = Model([comment_seq], output)
    print(model.summary())
    return model


def embedding(maxlen=50, max_features=4590, embed_size=16):
    # define the model
    model = Sequential()
    model.add(Embedding(max_features, embed_size, input_length=maxlen))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # summarize the model
    print(model.summary())
    return model

    
def model_fit(model,padded_docs,labels,train_index,test_index, epochs = 5):
    len_train = len(train_index)
    len_test = len(test_index)
    x_test =  np.zeros(shape = (len_test,len(padded_docs[0])))
    y_test = np.zeros(shape = (len_test))
    x_train = np.zeros(shape = (len_train,len(padded_docs[0])))
    y_train = np.zeros(shape = (len_train))
    i = 0
    for tri in train_index:
        x_train[i] = padded_docs[tri]
        y_train[i] = labels[tri]
        i+=1
    j = 0
    for ti in test_index:
        x_test[j] = padded_docs[ti]
        y_test[j] =  labels[ti]
        j +=1
        
    print("Test set from ",test_index[0], " to ",test_index[-1])

    model.fit(x_train, y_train,epochs=epochs)
    eva = model.evaluate(x_test,y_test)
    print('loss: ',eva[0])
    print('accuracy: ',eva[1])
    print('precision: ',eva[2])
    print('recall: ',eva[3])
    print('f1-score: ',(2*eva[3]*eva[2])/(eva[3]+eva[2]))
    return model 

def model_fit_no_test(model,x_train,y_train, epochs = 5):
    
    model.fit(x_train, y_train,epochs=epochs)
#     print('loss: ',eva[0])
#     print('accuracy: ',eva[1])
#     print('precision: ',eva[2])
#     print('recall: ',eva[3])
#     print('f1-score: ',(2*eva[3]*eva[2])/(eva[3]+eva[2]))
    return model 

import numpy
def predict_test(model, testData):
    test = numpy.reshape(testData,(1,testData.shape[0]))
    pr = model.predict(test)
    re = []
    if (pr >= 0.5):
        return ("yes")
    else:
        return ("no")
    
def predict_train(model, testData, truth):
    test = numpy.reshape(testData,(1,testData.shape[0]))
    pr = model.predict(test)
    print("==================")
    print("truth = ",truth )
    print("predict = ",pr )
    if (pr >= 0.5 and truth == 1) or (pr< 0.5 and truth ==0):
        print("true")
        return 1
    else:
        print("false")
        return 0
    
def write_result(predict, outPath,inpath ="assets/test_set_0520.csv"):
    dataframe_in = pd.read_csv(inpath, na_filter = False)
    dataframe_in["Expected"] = predict
    dataframe_in.to_csv(outPath)


In [40]:
# define documents
## read train Data
def read_train_Data():
    SPL = "<SPL>"
    path = "csv/trimedValues2.csv"
    train_file = path
    dataframe = pd.read_csv(path, na_filter = False)
    docs = []
    labels = []
    for i,data in dataframe.iterrows():
        d = data["comment"]
        c = data["code"]
        s = []
        s =  d+" "+SPL+" "+c # comment and code together
        docs.append(s)
        l = 1 if data["non-information"] == "yes" else 0
        labels.append(l)
    # integer encode the documents
    vocab_size = 2000
    encoded_docs = [one_hot(d, vocab_size) for d in docs]
    # pad documents to a max length of 41 words
    # The langest sentence contains 200 words but the Average is 21. If  padding all sentence to length of langest one, most of sentence
    # will be 0. That is not good for RNN, so we set padding length to 50
    summ = 0
    for i in docs:
        summ += len(i.split())
    print(summ/len(docs))
    # max_length = len(max(docs, key=len).split())
    max_length = 50
    ##
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return padded_docs, labels

In [51]:
# define documents
## read test Data
def read_test_Data():
    SPL = "<SPL>"
    tes_path = "csv/trimedValues3.csv"
    test_file = tes_path
    dataframe = pd.read_csv(test_file, na_filter = False)
    test_docs = []
    test_labels = []
    for i,data in dataframe.iterrows():
        d = data["comment"]
        c = data["code"]
        s = []
        s =  d+" "+SPL+" "+c # comment and code together
        test_docs.append(s)
    # integer encode the documents
    vocab_size = 2000
    test_encoded_docs = [one_hot(d, vocab_size) for d in test_docs]
    # pad documents to a max length of 41 words
    # The langest sentence contains 200 words but the Average is 21. If  padding all sentence to length of langest one, most of sentence
    # will be 0. That is not good for RNN, so we set padding length to 50
    summ = 0
    for i in test_docs:
        summ += len(i.split())
#     print(summ/len(test_docs))
    # max_length = len(max(docs, key=len).split())
    max_length = 50
    ##
    test_padded_docs = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
    return test_padded_docs

In [41]:
# train CNN with training data

padded_docs,labels = read_train_Data()
model_CNN = CNN(max_length,vocab_size,32)
model_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model_CNN = model_fit_no_test(model_CNN,padded_docs,labels)


21.7673531655225
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_seq (InputLayer)              (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 50, 32)       64000       x_seq[0][0]                      
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 49, 10)       650         embedding_10[0][0]               
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 48, 10)       970         embedding_10[0][0]               
___________________________________________________________________________

In [90]:
## CNN test
test_padded_docs = read_test_Data()
outPath = "result/CNN-Result.txt"
i = []
for tr in (test_padded_docs):
    i.append(predict_test(model_CNN,tr))
print("CNN Predicted non-informative ",i)
write_result(i, outPath)

CNN Predicted non-informative  ['no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 

In [56]:
# train RNN with training data
padded_docs,labels = read_train_Data()
model = RNN(max_length,vocab_size,16)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model = model_fit_no_test(model,padded_docs,labels, 20)

21.7673531655225
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 50, 16)            32000     
_________________________________________________________________
gru_4 (GRU)                  (None, 16)                1584      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 17        
Total params: 33,601
Trainable params: 33,601
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
1311/1311 [==============================] - 6s 5ms/step - loss: 0.6551 - accuracy: 0.6880 - precision_13: 0.2714 - recall_13: 0.0504
Epoch 2/20
1311/1311 [==============================] - 5s 4ms/step - loss: 0.6058 - accuracy: 0.7124 - precision_13: 0.0000e+00 - recall_13: 0.0000e+00
Epoch 3/20
1311/1311 [==============================] - 5s 4

In [91]:
## RNN test
test_padded_docs = read_test_Data()

i = []
for tr in (test_padded_docs):
    i.append(predict_test(model,tr))
print("RNN Predicted non-informative ",i)
outPath = "result/RNN-Result.txt"
write_result(i, outPath,)

RNN Predicted non-informative  ['no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no'

In [58]:
# train embedding with training data
padded_docs,labels = read_train_Data()
model_em = embedding(max_length,vocab_size,32)
model_em.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model_em = model_fit_no_test(model_em,padded_docs,labels)


21.7673531655225
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 50, 32)            64000     
_________________________________________________________________
flatten_18 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 1601      
Total params: 65,601
Trainable params: 65,601
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
1311/1311 [==============================] - 1s 640us/step - loss: 0.6330 - accuracy: 0.6972 - precision_14: 0.0000e+00 - recall_14: 0.0000e+00
Epoch 2/5
1311/1311 [==============================] - 0s 194us/step - loss: 0.5432 - accuracy: 0.7429 - precision_14: 0.7273 - recall_14: 0.1698
Epoch 3/5
1311/1311 [==============================] - 0s 

In [92]:
## embedding test
test_padded_docs = read_test_Data()

i = []
for tr in (test_padded_docs):
    i.append(predict_test(model_em,tr))
print("Embedding Predicted non-informative ",i)
outPath = "result/Embedding-Result.txt"
write_result(i, outPath)

Embedding Predicted non-informative  ['no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'n

In [ ]:
padded_docs, labels = read_train_Data()
# CNN train with split 
train_index = range(1050)
test_index = range(1050,len(padded_docs))
model_CNN = CNN(max_length,vocab_size,32)
model_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model_CNN = model_fit(model_CNN,padded_docs,labels,train_index,test_index)
# n_split=5
# for train_index,test_index in KFold(n_split).split(padded_docs):
#     # define the model
#     model = CNN(max_length,vocab_size,32)
    
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])

#     model = model_fit(model,padded_docs,labels,train_index,test_index)
## CNN test-train-result
i = 0
for f, b in zip(padded_docs[1050:], labels[1050:]):
    i += predict_train(model_CNN,f,b)
print("total num:",len(padded_docs[1050:]))
print(i)


# embedding train with split 
train_index = range(1050)
test_index = range(1050,len(padded_docs))
model_em = embedding(max_length,vocab_size,32)
model_em.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model_em = model_fit(model_em,padded_docs,labels,train_index,test_index)
i = 0
for f, b in zip(padded_docs[1050:], labels[1050:]):
    i += predict_train(model_em,f,b)
print("total num:",len(padded_docs[1050:]))
print(i)


# RNN train with split 
train_index = range(1050)
test_index = range(1050,len(padded_docs))
model = RNN(max_length,vocab_size,16)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model = model_fit(model,padded_docs,labels,train_index,test_index, 20)
## RNN train result
i = 0
for f, b in zip(padded_docs[1050:], labels[1050:]):
    i += predict_train(model,f,b)
print("total num:",len(padded_docs[1050:]))
print(i)